In [6]:
import keras as k
import keras.backend as K
from keras.layers import merge
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from keras.callbacks import History
from keras.layers import Activation
from keras.models import model_from_json
from keras.optimizers import Adam
from matplotlib import pyplot as plt
from scipy.ndimage import rotate as rot
from sklearn.model_selection import train_test_split
from keras import utils
import numpy as np
import tensorflow as tf
import cv2
import time
import pandas as pd

In [2]:
#load image data
all_data = np.load('images_dataset.npy')

###Just in case you want to try for few images, index all_data accordingly
X_test = all_data
print(X_test.shape)

def reset_tf_session():
    K.clear_session()
    tf.reset_default_graph()
    s = K.get_session()
    return s

from keras.models import load_model
s = reset_tf_session()

##Load saved model. Also spend time in tuning the model. This is a generic model we created from the training script.
autoencoder = load_model("hackathon_autoencoder.64-519.32.h5")  
encoder = autoencoder.layers[1]
decoder = autoencoder.layers[2]



def generate_embedding(img,encoder,decoder):
    """Inference the model to generate embedding"""
    start = time.time()
    code = encoder.predict(img[None])[0]  # Generated Embedding
    end = time.time()
    #print(end - start)
    reconstructed = decoder.predict(code[None])[0]
    
    return reconstructed, code

(3026, 128, 128, 3)


In [3]:
all_embeddings = list()
for i,row in enumerate(X_test):
    reconstruct, code = generate_embedding(row, encoder, decoder)
    all_embeddings.append(code)
embeddings_array = np.stack(all_embeddings)

In [20]:
def recommendProductsForUserId(UserID, totalRecommendations):
    #UserID = 26784
    global simArrPerUser,aggMergedDF,productAggData

    UserSpecificProducts = set(productAggData[productAggData["UserId"] == UserID]["productid"])
    
    allProducts = productMatrix[["productid"]].values
    UserIDs = aggMergedDF.index.values
    UserIDIndex = np.where(UserIDs == UserID)
    
    productIds = []
    productSortedSimilarity = np.argsort(-1 * simArrPerUser[UserIDIndex,:]).reshape(1,-1)
    
    #pdb.set_trace()
    for index in range(productSortedSimilarity.shape[1]):
        productIndex = productSortedSimilarity[0,index]
        if allProducts[productIndex][0] not in UserSpecificProducts:
            productIds.append(allProducts[productIndex][0])
    
    
    return productIds[0:totalRecommendations]

In [7]:
submissionDFUsers = pd.read_csv("../data/test.csv")
trainDFUsers = pd.read_csv("../data/train/train.csv")
productMatrix = pd.read_csv("product_image_id_list.csv",header=None)
productMatrix.columns = ["productid"]
productAggData = trainDFUsers.groupby(["productid","UserId"])["Quantity"].count().reset_index()
productAggData.columns = ["productid","UserId","PurchaseCount"]

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
productSimilarityMatrix = cosine_similarity(embeddings_array)

tmp = pd.concat([productMatrix[["productid"]]
           ,pd.DataFrame(productSimilarityMatrix)],axis = 1)

mergedDF = productAggData[["UserId","productid"]].merge(tmp, how = "inner",on = "productid", suffixes = "_y")
cols = mergedDF.columns[2:]
aggMergedDF = mergedDF.groupby("UserId")[cols].agg(['count','sum'])
aggMergedDF.columns = [str(col[0]) + "_" + col[1] for col in aggMergedDF.columns]
for col in cols:
    aggMergedDF[str(col) + "_sum"] = aggMergedDF[str(col) + "_sum"]/ aggMergedDF["0_count"]
aggMergedDF = aggMergedDF[[str(col) + "_sum" for col in cols]]
aggMergedDF.columns = [str(col).replace("_sum","") for col in aggMergedDF.columns]

simArrPerUser = np.asarray(aggMergedDF)

In [23]:
submission = pd.DataFrame()
for user in submissionDFUsers["UserId"].values:
    recommendations = "[" + ",".join(str(prod) for prod in recommendProductsForUserId(user,10)) + "]"
    submission = pd.concat([submission,pd.DataFrame({"UserId":[user],"product_list":[recommendations]})], axis = 0)

submission.to_csv("../Data/submission.csv",index= False)

In [21]:
recommendProductsForUserId(2,10)

[11659594,
 11659864,
 12658612,
 12658288,
 11714682,
 12658502,
 11659614,
 12947668,
 11659570,
 12657900]